In [1]:
# !pipenv install tensorflow-macos tensorflow
# !pipenv install keras

In [2]:
import numpy as np
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU') # 我的電腦不下這條會有些奇奇怪怪的問題

In [3]:
with open("X_y_keras.npz", "rb") as file:
    npz = np.load(file)
    X = npz["X"]
    y = npz["y"]

In [4]:
X.shape, y.shape

((75934, 14, 11), (75934, 7, 2))

我個理解是
X.shape = (75934, 14, 11) 分別是資料筆數，lag 以及 feature 數目
y.shape =  (75934, 7, 2) 分別是資料筆數，step 以及兩個依變數(fare, remaining_seats)
應該沒錯吧（汗）所以依變項其實是二維的

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Lambda, Reshape
import keras.backend as K

# 定義調整形狀的函數
def adjust_shape(x):
    return x[:, :7, :2]


In [6]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(14, 11), return_sequences=True))  # LSTM
model.add(TimeDistributed(Dense(units=2)))  # 輸出層

# 使用 Lambda 層來調整形狀
model.add(Lambda(adjust_shape))

model.compile(optimizer='adam', loss='mse')

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 64)            19456     
                                                                 
 time_distributed (TimeDist  (None, 14, 2)             130       
 ributed)                                                        
                                                                 
 lambda (Lambda)             (None, 7, 2)              0         
                                                                 
Total params: 19586 (76.51 KB)
Trainable params: 19586 (76.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
2373/2373 [==============================] - 7s 3ms/step - loss: 70311.1562
Epoch 2/10
2373/2373 [==============================] - 6s 3ms/step - loss: 40181.0156
Epoch 3/10
2373/2373 [==============================] - 6s 3ms/step - loss: 20703.0430
Epoch 4/10
2373/2373 [==============================] - 6s 3ms/step - loss: 10304.1426
Epoch 5/10
2373/2373 [==============================] - 6s 3ms/step - loss: 6206.7114
Epoch 6/10
2373/2373 [==============================] - 6s 3ms/step - loss: 4941.7993
Epoch 7/10
2373/2373 [==============================] - 6s 3ms/step - loss: 4501.1343
Epoch 8/10
2373/2373 [==============================] - 6s 3ms/step - loss: 4229.6787
Epoch 9/10
2373/2373 [==============================] - 6s 3ms/step - loss: 3823.3650
Epoch 10/10
2373/2373 [==============================] - 6s 3ms/step - loss: 3151.8611


Ｊ個 Loss 也太噁了吧（汗